<img src="https://www.uc3m.es/ss/Satellite?blobcol=urldata&blobkey=id&blobtable=MungoBlobs&blobwhere=1371573952659">


---


# **WEB ANALYTICS COURSE 4 - SEMESTER 2**
# **BACHELOR IN DATA SCIENCE AND ENGINEERING**

# **LAB 1 APIs - WORLD BANK**


Made by:

### Marina Gómez Rey (100472836)
### María Ángeles Magro Garrote (100472867)
### Sara Piñas García (100472784)
### Eduardo González Agüero (100472704)

# 0. LAB PREPARATION

Students have to complete the following tasks before attedning the lab:

1. **Read and study the API documentation to have some initial notions of the functionality of the World Bank API. Following, we share several links to the documentation related to the World Bank API:**
- https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures
- https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information
- https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation

2. **The key element of the World Bank API are the "indicators". Next, we share a link that may simplify the search of indicators through a search tool. Once you have selected an indicator you can find its codification within the url bar of the browser.**

- https://data.worldbank.org/indicator?tab=featured

# **1. INTRODUCTION**

* The goal of this lab is to gain experience testing a widely-used API such as the World Bank API that includes bunch of information about countries indicators in economy, health, education, agriculture, etc.

* The lab includes 5 milestones that will drive the student through the use of several indicators.  

* The lab will be done in groups of 23 students.

* The lab will use two complete consecutive sessions (4 hours). The students are expected to complete the 5 milestones proposed in the lab within these 2 sessions

* **The final mark will be computed as a function of the number of milestones successfully completed.**

* **Each group should also upload their lab notebook in the corresponding task in Aula Global.**

* Upon completing all the milestones, students should call the professor, who will check the correctness of the solution. Partial milestones checks may be allowed in some cases.

# 2. **MILESTONES**

In this section we describe one by one the milestones and leave a space to the students to implement the code to complete the requested task.

**NOTE: Unless otherwise stated, all the milestones have to deliver information about countries. Therefore, you should not consider regions or any othre aggreated information in your analysis.**

# **2.1. MILESTONE 1: POPULATION**:
Retrieve the 2022 countries' population and show the Top 10 countries and the Bottom 10 countries within the World Bank database.



In [1]:
import requests
import pandas as pd

# URL para obtener la lista de países
url = "https://api.worldbank.org/v2/country/all?format=json&per_page=296"

# Hacer la petición para obtener los países
countries = requests.get(url).json()[1]

# Filtrar solo los códigos de los países válidos (excluir agregados)
country_codes = [
    c['id'] for c in countries
    if c['region']['value'] != "Aggregates" and c["capitalCity"] != "" and ")" not in c["name"]
]

# URL para obtener datos de población en 2022 para todos los países
pop_url = f"https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2022&format=json&per_page=30000"

# Hacer la petición para obtener los datos de población
pop_response = requests.get(pop_url).json()

# Filtrar solo los datos de población para los códigos de país válidos
population_data = [
    {
        'country_code': entry['countryiso3code'],
        'country_name': entry['country']['value'],
        'population': entry['value']
    }
    for entry in pop_response[1]
    if entry['countryiso3code'] in country_codes
]

# Crear DataFrame, eliminar valores nulos y ordenar por población
df = pd.DataFrame(population_data).dropna().sort_values(by='population', ascending=False)

# Mostrar los 10 países con mayor y menor población
print("Top 10 Countries by Population in 2022:\n", df.head(10))
print("\nBottom 10 Countries by Population in 2022:\n", df.tail(10))


Top 10 Countries by Population in 2022:
     country_code        country_name  population
86           IND               India  1417173173
40           CHN               China  1412175000
199          USA       United States   333271411
87           IDN           Indonesia   275501339
144          PAK            Pakistan   235824862
139          NGA             Nigeria   218541212
26           BRA              Brazil   215313498
15           BGD          Bangladesh   171186372
156          RUS  Russian Federation   144236933
122          MEX              Mexico   127504125

Bottom 10 Countries by Population in 2022:
     country_code              country_name  population
193          TCA  Turks and Caicos Islands       45703
3            ASM            American Samoa       44273
119          MHL          Marshall Islands       41569
110          LIE             Liechtenstein       39327
125          MCO                    Monaco       36469
159          SMR                San Marino   

# **2.2. MILESTONE 2: WOMEN Vs. MEN POPULATION**:
Obtain the % of men and women for each country and compute the difference among them using the formula %women - %men. Display:

1- The number of countries with more women than men.

2- The number of countries with more men than women

3- The 10 countries with more women compared to men (ten countries with the largest positive value of the previous metric)

- The 10 countries with more men compared to women (ten countries with the largest negative value of the previous metric).

**Note**: You can use the indicator the absolute number of men and women from the World Bank API and compute the % for each country and the difference, or you can use the indicator given directly the %.



In [2]:
import requests
import pandas as pd

# URL para obtener la lista de países
url = "https://api.worldbank.org/v2/country/all?format=json&per_page=296"

# Hacer la petición para obtener los países
countries = requests.get(url).json()[1]

# Filtrar solo los códigos de los países válidos (excluir agregados)
country_codes = [
    c['id'] for c in countries
    if c['region']['value'] != "Aggregates" and c["capitalCity"] != "" and ")" not in c["name"]
]

# URL para obtener datos de población por género en 2022 para todos los países
pop_gender_url = f"https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL.FE.ZS?date=2022&format=json&per_page=30000"

# Hacer la petición para obtener los datos de población por género (mujeres)
response_women = requests.get(pop_gender_url).json()[1]

# URL para obtener datos de población por género en 2022 para hombres
pop_gender_url_men = f"https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL.MA.ZS?date=2022&format=json&per_page=30000"

# Hacer la petición para obtener los datos de población por género (hombres)
response_men = requests.get(pop_gender_url_men).json()[1]

# Crear un DataFrame para almacenar los datos
gender_data = []

# Recopilar datos de hombres y mujeres
for entry_women in response_women:
    if entry_women['countryiso3code'] in country_codes:
        gender_data.append({
            'country_name': entry_women['country']['value'],
            'percent_women': entry_women['value'],
            'percent_men': next(
                (entry_men['value'] for entry_men in response_men if entry_men['countryiso3code'] == entry_women['countryiso3code']),
                None
            )
        })

# Crear DataFrame
df_gender = pd.DataFrame(gender_data)

# Calcular la diferencia en los porcentajes
df_gender['difference'] = df_gender['percent_women'] - df_gender['percent_men']

# Contar países con más mujeres que hombres (valor positivo)
countries_more_women = df_gender[df_gender['difference'] > 0].shape[0]
print(f"Number of countries with more women than men: {countries_more_women}")

# Contar países con más hombres que mujeres (valor negativo)
countries_more_men = df_gender[df_gender['difference'] < 0].shape[0]
print(f"Number of countries with more men than women: {countries_more_men}")

# Ordenar para obtener el top 10 en ambos casos
top_10_more_women = df_gender.sort_values(by='difference', ascending=False).head(10)
print("Top 10 countries with more women than men:")
print(top_10_more_women[['country_name', 'difference']])

top_10_more_men = df_gender.sort_values(by='difference', ascending=True).head(10)
print("Top 10 countries with more men than women:")
print(top_10_more_men[['country_name', 'difference']])


Number of countries with more women than men: 128
Number of countries with more men than women: 80
Top 10 countries with more women than men:
           country_name  difference
8               Armenia   10.081429
196             Ukraine    8.161847
17              Belarus    7.912183
105              Latvia    7.230537
156  Russian Federation    7.115055
111           Lithuania    6.063042
71              Georgia    5.992546
9                 Aruba    5.729720
153         Puerto Rico    5.661284
152            Portugal    5.632719
Top 10 countries with more men than women:
                 country_name  difference
154                     Qatar  -45.020003
197      United Arab Emirates  -38.706801
14                    Bahrain  -23.920466
143                      Oman  -21.735691
102                    Kuwait  -21.622147
161              Saudi Arabia  -15.242590
116                  Maldives  -15.052684
141  Northern Mariana Islands   -7.505398
22                     Bhutan   -5.802890

## **2.3. MILESTONE 3: GDP PER CAPITA ACCORDING FOR INCOME LEVEL GROUPS**:

Compute the average increase/decrease in percentage for the GDP per capita in US dollars in the following two periods: 2000-2022 and  2010-2022, GDPfor the following income groups: low-income economies, lower-middle-income economies, middle economies, upper-middle-income economies and high-income economies. The following, link provides information of the different country aggregations carried out by the World Bank.  

https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

 You should compute the %GDP increase as follows. Given country A with a PIB Per Capita \$20000 in 2000 and \$30000 in 2022 the increase/decrease should be computed as follow:

%GDP increase = 100*(30000-20000)/20000=50%.


In [3]:
income_groups = ['LIC', 'LMC', 'MIC', 'UMC', 'HIC']  # Low-income, Lower-middle-income, Middle-income, Upper-middle-income, High-income

# We get the new url taking the GPD and the defined groupe and take the years that are asked
def get_gdp_data(group):
    url = f"https://api.worldbank.org/v2/country/{group}/indicator/NY.GDP.PCAP.CD?date=2000:2022&format=json&per_page=1000"
    response = requests.get(url)
    data = response.json()[1]
    df = pd.DataFrame(data)[['country', 'date', 'value']]
    df['country'] = df['country'].apply(lambda x: x['value'])

    # We take the numeric value to take the gpd data
    df['value'] = pd.to_numeric(df['value'])
    return df

# Function to calculate the change in the gpd with the start and end years
# as we have two different periods
def calculate_gdp_change(df, start_year, end_year):
    df_start = df[df['date'] == str(start_year)]
    df_end = df[df['date'] == str(end_year)]

    # To differentiate the start and end years, we add the suffixes
    df_combined = pd.merge(df_start, df_end, on='country', suffixes=('_start', '_end'))

    # We apply the given formula to calculate the change in the GPD
    df_combined['gdp_change'] = 100 * (df_combined['value_end'] - df_combined['value_start']) / df_combined['value_start']

    # We return the mean of that group
    return df_combined['gdp_change'].mean()

# Calculate the change in the countries with the countries in the income_groups to
# not take the aggregates and save them in a list
results = []
for group in income_groups:
    df = get_gdp_data(group)

    # Define the two given periods and calculate the percentage of change with the funct
    for period in [(2000, 2022), (2010, 2022)]:
        avg_change = calculate_gdp_change(df, period[0], period[1])
        results.append({'Income Group': group, 'Period': f"{period[0]}-{period[1]}", 'Avg GDP % Change': avg_change})

results_df = pd.DataFrame(results)
print(results_df)

  Income Group     Period  Avg GDP % Change
0          LIC  2000-2022        132.798987
1          LIC  2010-2022          3.285398
2          LMC  2000-2022        333.581013
3          LMC  2010-2022         62.417742
4          MIC  2000-2022        397.065083
5          MIC  2010-2022         70.792248
6          UMC  2000-2022        451.376997
7          UMC  2010-2022         79.266182
8          HIC  2000-2022        106.967102
9          HIC  2010-2022         29.770288


# **2.4. MILESTONE 4: TOP 5 COUNTRIES INCREASE GDP PER INCOME-GROUP**

For each of the income groups included in Milestone 3 and the period 2010-2022 list the Top 5 countries in terms of %GDPR per capita increase along with the value

**NOTE**: Do not consider the countries for which you do not have data either in 2010 or 2022 or both of them

In [4]:
# Function to filter countries by income group
def filter_countries_by_income_group(countries_data, income_group):
    # Filter countries by region and capital city conditions
    country_codes = [c['id'] for c in countries_data if c['region']['value'] != "Aggregates"
                     and c["capitalCity"] != "" and ")" not in c["name"]]
    # Further filter by income group
    filtered_countries = [country for country in country_codes
                          if next(c for c in countries_data if c['id'] == country)['incomeLevel']['id'] == income_group]
    return filtered_countries


# Function to get GDP per capita data for an income group
def get_gdp_data(filtered_country_codes):
    if not filtered_country_codes:  # Check if there are any country codes
        return pd.DataFrame()  # Return an empty DataFrame if no countries
    country_codes_str = ';'.join(filtered_country_codes)  # Join country codes into a single string
    url = f"https://api.worldbank.org/v2/country/{country_codes_str}/indicator/NY.GDP.PCAP.CD?date=2010:2022&format=json&per_page=1000"
    response = requests.get(url)
    data = response.json()[1]
    df = pd.DataFrame(data)[['country', 'date', 'value']]
    df['country'] = df['country'].apply(lambda x: x['value'])
    return df

# Function to get the Top 5 countries with the largest percentage change in GDP per capita
def get_top_5_countries(df, start_year, end_year):
    if df.empty:  # Check if DataFrame is empty
        return pd.DataFrame()  # Return an empty DataFrame if no data
    df_start = df[df['date'] == str(start_year)]
    df_end = df[df['date'] == str(end_year)]
    df_combined = pd.merge(df_start, df_end, on='country', suffixes=('_start', '_end'))

    # Calculate the percentage change in GDP per capita
    df_combined['gdp_change'] = 100 * (df_combined['value_end'] - df_combined['value_start']) / df_combined['value_start']

    # Sort by percentage change in descending order
    df_combined = df_combined.sort_values(by='gdp_change', ascending=False)

    # Select the Top 5 countries with the largest increase
    top_5 = df_combined[['country', 'gdp_change']].head(5)

    return top_5

# -------------------- MAIN FUNCTION CALLS --------------------

# Get the results for each income group
top_5_results = {}

for group in income_groups:
    # For each income group, get the filtered countries and their GDP data
    filtered_countries = filter_countries_by_income_group(countries, group)

    if not filtered_countries:  # Check if no countries were filtered
        print(f"No countries found for income group: {group}")
        continue  # Skip to the next income group if no countries found

    df = get_gdp_data(filtered_countries)

    if df.empty:  # Check if the GDP data is empty
        print(f"No GDP data found for income group: {group}")
        continue  # Skip to the next income group if no GDP data available

    top_5_countries = get_top_5_countries(df, 2010, 2022)  # Get the Top 5 for 2010-2022

    if top_5_countries.empty:  # Check if the top 5 result is empty
        print(f"No top 5 countries found for income group: {group}")
        continue  # Skip to the next income group if no top 5 found

    top_5_countries['Income Group'] = group  # Add the income group
    top_5_results[group] = top_5_countries  # Store in the dictionary by group

# Display the results
for group, top_5 in top_5_results.items():
    print(f"Top 5 countries with the largest increase in GDP per capita for the {group} income group (2010-2022):")
    print(top_5)
    print("\n")

No countries found for income group: MIC
Top 5 countries with the largest increase in GDP per capita for the LIC income group (2010-2022):
             country  gdp_change Income Group
6           Ethiopia  206.315061          LIC
19           Somalia  164.937788          LIC
4   Congo, Dem. Rep.  104.598773          LIC
16            Rwanda   62.690758          LIC
9            Liberia   51.811254          LIC


Top 5 countries with the largest increase in GDP per capita for the LMC income group (2010-2022):
        country  gdp_change Income Group
2    Bangladesh  246.006100          LMC
41  Timor-Leste  194.895984          LMC
10     Djibouti  166.988355          LMC
45     Viet Nam  148.158110          LMC
32        Nepal  128.824895          LMC


Top 5 countries with the largest increase in GDP per capita for the UMC income group (2010-2022):
             country  gdp_change Income Group
9              China  178.269546          UMC
30           Moldova  134.540452          UMC
2

# **2.5. MILESTONE 5: CO2 emission per capita**

Retrieve the most recent non empty value for the amount of CO2 emission per capita (metric tons per country) for all the countries. Display the 30 countries with the highest CO2 emission per capita along with their value and the year related to that value.

**NOTE**: You cannot search manually the year and use it in your query for this milestone.


In [5]:
import requests
import pandas as pd

co2data = []

co2_url = f"https://api.worldbank.org/v2/country/all/indicator/EN.ATM.CO2E.PC?format=json&mrnev=1&per_page=1000"
response = requests.get(co2_url).json()


# Check if data is returned
for entry in response[1]:
        # Append CO2 data for each country to the list
        if entry['countryiso3code'] in country_codes:
          co2data.append({
              'country_name': entry['country']['value'],
              'co2_per_capita': entry['value'],
              'year': entry['date']
          })

# We sort the df and get the firt 30
co2df = pd.DataFrame(co2data).sort_values(by='co2_per_capita', ascending=False).head(30)

print("Top 30 countries with the highest CO2 emissions per capita:")
print(co2df)

Top 30 countries with the highest CO2 emissions per capita:
             country_name  co2_per_capita  year
138                 Qatar       31.726842  2020
12                Bahrain       21.976908  2020
24      Brunei Darussalam       21.705812  2020
90                 Kuwait       21.169610  2020
179  United Arab Emirates       20.252272  2020
128                  Oman       15.636201  2020
8               Australia       14.776137  2020
144          Saudi Arabia       14.266585  2020
31                 Canada       13.591696  2020
181         United States       13.032222  2020
100            Luxembourg       12.456953  2020
85             Kazakhstan       11.297743  2020
140    Russian Federation       11.141653  2020
89            Korea, Rep.       10.990030  2020
175          Turkmenistan       10.184086  2020
172   Trinidad and Tobago       10.157119  2020
130                 Palau        8.802582  2020
45                Czechia        8.304017  2020
83                  Japan   